In [1]:
import pandas as pd
import requests
import folium
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

# Scraping Wikipedia

We read from the HTML table using pandas directly. This returns a list of dataframes, and we are interested in the first dataframe. We take only the rows of the dataframe where `'Borough'` is _not_ `'Not assigned'`. We reset the index numbering.

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, header=0)[0]
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


At this point, the dataframe looks almost right, but the instructions imply that some postal codes may be listed twice. Let's check if any postal code appears more than once in the dataframe:

In [3]:
(pd.value_counts(df['Postal code']) >1).any()

False

Great, now we need to make sure the Neighborhoods are separated by commas instead of forward slashes.

In [4]:
df['Neighborhood'] = df['Neighborhood'].str.replace(' /', ',')
df.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now we need to check for assigned boroughs with "Not assigned" neighborhoods.

In [5]:
(df['Neighborhood'] == 'Not assigned').any()

False

There do not appear to be any. Apparently the dataset has changed since the assignment was written.

# Get Latitude and Longitude

Now we load and inspect the latitude and longitude data from the provided csv file.

In [6]:
!wget --quiet http://cocl.us/Geospatial_data -O lat_lng.json

In [7]:
lat_lng = pd.read_csv('lat_lng.json')
lat_lng.rename(columns={'Postal Code': 'Postal code'}, inplace=True)

In [8]:
lat_lng

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Merge the two dataframes.

In [9]:
df = pd.merge(df, lat_lng)
df

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Exploring and clustering neighborhods

First, we take only boroughs containing the word "Toronto"

In [10]:
df = df[df['Borough'].str.contains('Toronto')]
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


Let's create a map of the neighborhoods. We use geopy to find the latitude and longitude of Toronto

In [11]:
geolocator = Nominatim(user_agent='toronto')
location = geolocator.geocode('Toronto, Canada')
latitude, longitude = location.latitude, location.longitude

latitude, longitude

(43.6534817, -79.3839347)

And we create a map using folium

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neigh in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{} / {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        parse_html=False).add_to(map_toronto)  
map_toronto

Now let's use Foursquare to explore the Neighborhoods. We define our credentials.

In [13]:
CLIENT_ID = 'ESQILQUJ45MEFP5CYAUTQAPRNT1YA22FASZDU0VCI5VIE2EP'
CLIENT_SECRET = 'MZIKSDQUL4OHRIPALRKSUQSKUMMVKM2PCGAMGJPWKJ3G0MBZ'
VERSION = '20180605' # Foursquare API version
LIMIT = 100

Define a function to get venues nearby each neighborhood.

In [14]:
def get_nearby(names, lats, lngs, radius=500):
    venues_list = []
    for name, lat, lng in zip(names, lats, lngs):
#         print(name)
        
        # Create API request
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
venues = get_nearby(names=df['Neighborhood'],
                    lats=df['Latitude'],
                    lngs=df['Longitude']
                                  ) # this may take ~30 seconds

In [16]:
venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Let's use one-hot encoding for the venue categories and make the first column `'Neighborhood'`.

In [17]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot.drop(columns=['Neighborhood'], inplace=True)
venues_onehot.insert(0, 'Neighborhood', venues['Neighborhood'])
venues_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Now group by neighborhood:

In [75]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.0,0.0,0.017857,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
2,Business reply mail Processing CentrE,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.055556
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013699,0.0,...,0.0,0.0,0.013699,0.0,0.0,0.013699,0.0,0.0,0.0,0.013699


Perform $k$-means to cluster the neighborhood. In playing around with $k$, I found that $k>3$ ended up splitting off individual neighborhoods into clusters alone.

In [76]:
k = 3
kmeans = KMeans(n_clusters=k, random_state=0) \
               .fit(venues_grouped.drop('Neighborhood', 1))

We will get the top N venues for each neighborhood and store them in the columns V1...VN.

In [77]:
N = 5

# make N new columns
for i in range(N):
    venues_grouped['V{}'.format(i+1)] = ''

In [78]:
venues_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,V1,V2,V3,V4,V5
0,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,,,,,
1,"Brockton, Parkdale Village, Exhibition Place",0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,,,,,
2,Business reply mail Processing CentrE,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.055556,,,,,
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.0625,0.0625,0.0625,0.125,0.125,0.0625,0.000000,0.0,...,0.000000,0.0,0.0,0.0,0.000000,,,,,
4,Central Bay Street,0.0,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.013699,0.0,...,0.013699,0.0,0.0,0.0,0.013699,,,,,


In [79]:
for i in range(venues_grouped.shape[0]): # loop over rows
    v = venues_grouped.iloc[i, 1:-N] # get each row (skipping 'Neighborhood' column and new empty columns)
    v.sort_values(ascending=False, inplace=True) # sort in descending order
    for j in range(N): # get top N venues
        venues_grouped.loc[venues_grouped.index[i], 'V{}'.format(j+1)] = v.index[j]

Extract a dataframe with just the top N venues for each neighborhood.

In [80]:
v_topN = venues_grouped[['Neighborhood', *['V{}'.format(j+1) for j in range(N)]]]
v_topN.head()

,Neighborhood,V1,V2,V3,V4,V5
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Café
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium
2,Business reply mail Processing CentrE,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Sculpture Garden,Harbor / Marina,Bar
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant


Add the cluster label to the dataframe of the top N venues.

In [81]:
v_topN.insert(1, 'Cluster label', kmeans.labels_)
v_topN.head()

,Neighborhood,Cluster label,V1,V2,V3,V4,V5
0,Berczy Park,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Café
1,"Brockton, Parkdale Village, Exhibition Place",0,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium
2,Business reply mail Processing CentrE,0,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Burrito Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,Airport Lounge,Airport Service,Sculpture Garden,Harbor / Marina,Bar
4,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant


Now let's merge this with the dataframe with latitute and longitude information.

In [82]:
v_topN = v_topN.merge(df, on='Neighborhood')
v_topN.head()

,Neighborhood,Cluster label,V1,V2,V3,V4,V5,Postal code,Borough,Latitude,Longitude
0,Berczy Park,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Café,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,M6K,West Toronto,43.636847,-79.428191
2,Business reply mail Processing CentrE,0,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Burrito Place,M7Y,East Toronto,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,Airport Lounge,Airport Service,Sculpture Garden,Harbor / Marina,Bar,M5V,Downtown Toronto,43.628947,-79.394420
4,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,M5G,Downtown Toronto,43.657952,-79.387383


We will add the cluster label as a color on the map

In [83]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set up color scheme
rainbow = [colors.rgb2hex(i) for i in cm.rainbow(np.linspace(0,1,k))]

for lat, lng, cluster, neigh in zip(v_topN['Latitude'], v_topN['Longitude'], v_topN['Cluster label'], v_topN['Neighborhood']):
    label = '{} / {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        parse_html=False).add_to(map_clusters)  
map_clusters

We see that most of the neighborhoods are clustered together, while there is one neighborhood in a cluster by itself, and 3 neighborhoods in a third cluster.

The first cluster appears to have Coffee Shops, Cafes, and other restaurants as the most common venues:

In [84]:
v_topN[v_topN['Cluster label']==0]

,Neighborhood,Cluster label,V1,V2,V3,V4,V5,Postal code,Borough,Latitude,Longitude
0,Berczy Park,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Café,M5E,Downtown Toronto,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,M6K,West Toronto,43.636847,-79.428191
2,Business reply mail Processing CentrE,0,Light Rail Station,Yoga Studio,Pizza Place,Auto Workshop,Burrito Place,M7Y,East Toronto,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0,Airport Lounge,Airport Service,Sculpture Garden,Harbor / Marina,Bar,M5V,Downtown Toronto,43.628947,-79.394420
4,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Middle Eastern Restaurant,M5G,Downtown Toronto,43.657952,-79.387383
5,Christie,0,Grocery Store,Café,Park,Diner,Baby Store,M6G,Downtown Toronto,43.669542,-79.422564
6,Church and Wellesley,0,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Sushi Restaurant,M4Y,Downtown Toronto,43.665860,-79.383160
7,"Commerce Court, Victoria Hotel",0,Coffee Shop,Café,Restaurant,Hotel,Gym,M5L,Downtown Toronto,43.648198,-79.379817
8,Davisville,0,Dessert Shop,Sandwich Place,Café,Coffee Shop,Italian Restaurant,M4S,Central Toronto,43.704324,-79.388790
9,Davisville North,0,Department Store,Food & Drink Shop,Park,Gym,Breakfast Spot,M4P,Central Toronto,43.712751,-79.390197


The next cluster is unique in that all three neighborhoods have a park listed as the top venue. In fact, Lawrence Park and Rosedale are close to large parks on the map. It is also rare to see restaurants in the top 5.

In [85]:
v_topN[v_topN['Cluster label']==1]

,Neighborhood,Cluster label,V1,V2,V3,V4,V5,Postal code,Borough,Latitude,Longitude
12,Forest Hill North & West,1,Park,Jewelry Store,Trail,Sushi Restaurant,Bus Line,M5P,Central Toronto,43.696948,-79.411307
18,Lawrence Park,1,Park,Swim School,Bus Line,Yoga Studio,Diner,M4N,Central Toronto,43.728020,-79.388790
26,Rosedale,1,Park,Playground,Trail,Yoga Studio,Department Store,M4W,Downtown Toronto,43.679563,-79.377529


The final cluster has one neighborhood that surprisingly has a restaurant listed as the top venue, so at first glance one might think this should belong to the first cluster. However, the lack of a Coffee Shop or Cafe and presence of unique top venues (like Event Space and Electronics Store) may have caused this neighborhood to fall into its own cluster.

In [87]:
v_topN[v_topN['Cluster label']==2]

,Neighborhood,Cluster label,V1,V2,V3,V4,V5,Postal code,Borough,Latitude,Longitude
20,"Moore Park, Summerhill East",2,Restaurant,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,M4T,Central Toronto,43.689574,-79.38316
